In [6]:
import os

In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

# Make all the layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers:
    layer.trainable = False
  # Print the model summary
pre_trained_model.summary()

'wget' is not recognized as an internal or external command,
operable program or batch file.


OSError: Unable to open file (unable to open file: name = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [46]:
class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
            if(logs.get('val_acc')>0.75 or logs.get('acc')>0.80):
                self.model.stop_training = True

In [47]:
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [48]:
td='/media/izaya/Windows/train_facereg/'
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,)
vd='/media/izaya/Windows/validation_facereg/'
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [49]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(last_output),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = tf.keras.layers.Dense(3, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

model.summary()

In [ ]:
model.summary()

In [50]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [51]:
train_generator = training_datagen.flow_from_directory(
   td,
    target_size=(150,150),
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    vd,
    target_size=(150,150),
    class_mode='categorical'
)

Found 1525 images belonging to 4 classes.
Found 99 images belonging to 4 classes.


In [52]:
hi = model.fit_generator(train_generator, epochs=12, validation_data = validation_generator, verbose = 1)

Epoch 1/12
48/48 [==============================] - 33s 685ms/step - loss: 1.5797 - acc: 0.3548 - val_loss: 1.3480 - val_acc: 0.4545
Epoch 2/12
48/48 [==============================] - 33s 679ms/step - loss: 1.0704 - acc: 0.5652 - val_loss: 1.0266 - val_acc: 0.4949
Epoch 3/12
48/48 [==============================] - 33s 682ms/step - loss: 0.8100 - acc: 0.6925 - val_loss: 0.4359 - val_acc: 0.9192
Epoch 4/12
48/48 [==============================] - 33s 682ms/step - loss: 0.6091 - acc: 0.7810 - val_loss: 0.2173 - val_acc: 1.0000
Epoch 5/12
48/48 [==============================] - 33s 686ms/step - loss: 0.4565 - acc: 0.8400 - val_loss: 0.1019 - val_acc: 1.0000
Epoch 6/12
48/48 [==============================] - 32s 675ms/step - loss: 0.3654 - acc: 0.8662 - val_loss: 0.0515 - val_acc: 1.0000
Epoch 7/12
48/48 [==============================] - 33s 684ms/step - loss: 0.2563 - acc: 0.9089 - val_loss: 0.0113 - val_acc: 1.0000
Epoch 8/12
48/48 [==============================] - 33s 685ms/step - 

In [53]:
model.save('mahi_facerec.h5')

In [54]:
from tensorflow.keras.models import load_model
from keras_preprocessing import image

In [55]:
import numpy as np

In [56]:
model=load_model('mahi_facerec.h5')
test_image = image.load_img('/home/izaya/Downloads/tejuu/test1/IMG_20200311_164919.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
model.predict(test_image, batch_size=1)

array([[0., 1., 0., 0.]], dtype=float32)